In [9]:
import os
import zipfile
import PyArabic
import logging

In [2]:
def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    local_filename = os.path.join('../', filename)

    statinfo = os.stat(local_filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + local_filename +
                        '. Can you get to it with a browser?')
    return local_filename


filename = maybe_download('webteb.zip',  81750261)

Found and verified webteb.zip


In [4]:
# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
        data = f.read(f.namelist()[0]).decode('utf-8').split()
    return data


data = read_data(filename)
print('Data size', len(set(data)))

Data size 627435


In [5]:
def preprocess_word(word):
    preprocess = PyArabic.ArabicPreprocessor()
    word = preprocess.deNoise(word)
    return word

def process_data(data):
    with open('stopwords.txt', 'r', encoding='utf-8') as f:
        stopwords = f.readlines()

    data = [word for word in data if word not in stopwords]
    
    return data

data = process_data(data)

In [10]:
import evaluate
import glove


# Mock corpus (shamelessly stolen from Gensim word2vec tests)
test_corpus = ' '.join(data)

glove.logger.setLevel(logging.INFO)
vocab = glove.build_vocab(test_corpus)
cooccur = glove.build_cooccur(vocab, test_corpus, window_size=10)
id2word = evaluate.make_id2word(vocab)

W = glove.train_glove(vocab, cooccur, vector_size=10, iterations=500)

# Merge and normalize word vectors
W = evaluate.merge_main_context(W)



INFO:glove:Building vocab from corpus


KeyboardInterrupt: 

In [6]:
data[:10]

['#',
 "'فقدانُ",
 'الوزنِ',
 'شعورٌ',
 "رائعٌ'",
 '__شارك',
 '__غرد',
 'ازداد',
 'وزنُ',
 'نيك']

In [ ]:
W